In [79]:
import struct, inspect
import hexdump, zlib
import traceback

def decode_1(body):
    key = bytearray(b'\xf9\xa3\xff\xf8')
    ret = bytearray(body)
    ## 0F8FFA3F9h
    for i in range(len(body)):
        ret[i] = ret[i] ^ key[i%4]
    return bytes(ret)

def decode_2(body):
    compressed_len = struct.unpack('i', body[8:12])[0]
#     key = bytearray(b'\xf9\xa3\xff\xf8')
    key = struct.unpack('i', b'\xf9\xa3\xff\xf8')[0]
#     key = bytearray(struct.pack('i', 0xf8ffa3f9 - compressed_len))
    key = bytearray(struct.pack('i', key - compressed_len))
    ret = bytearray(body)
    ## 0F8FFA3F9h
    for i in range(len(body)):
        ret[i] = ret[i] ^ key[i%4]
    for i in (8, 9, 10, 11):
        ret[i] = ret[i] ^ key[i%4]
    return bytes(ret)

def decode_block_header(body):
    origin_len, compressed_len = struct.unpack('ii', body[8:16])
    key = struct.unpack('i', b'\xf9\xa3\xff\xf8')[0]
    compressed_len = compressed_len ^ key - origin_len
    key = struct.pack('i', key - origin_len)
    return key, compressed_len, origin_len

def decode_block_bak(body, key):
    ret = bytearray(body)
    key = bytearray(key)
#     end = ((len(body) - 4) >> 2) << 2  # last 4 bytes never encode
#     end_2 = (len(body) >> 4) << 4
    end = ((len(body) ) >> 2) << 2
    if end_2 > end:
        end = end_2
    for i in range(end):
        ret[i] = ret[i] ^ key[i%4]
    return ret

def decode_block_bak_2(body, key):
    ret = bytearray(body)
    key = bytearray(key)
    a2 = len(body) + 16
    v13 = a2 - 4;
    v15 = (a2 - 21) >> 2;
    v16 = v15 + 1;
    v17 = ((v15 - 3) >> 2) + 1;
    v18 = 4 * v17;
    v23 = 16 * v17 + 16;
    if v16 == v18:
        end = 16*v17
    else:
        end = 16*v17 + 4
        if v23 + 4 < v13:
            end += 4
            if v23 + 8 < v13:
                end += 4
    for i in range(end):
        ret[i] = ret[i] ^ key[i%4]
    return ret

def decode_block(body, key):
    ret = bytearray(body)
    key = bytearray(key)
    end = (len(body)-1) >> 2 << 2
    for i in range(end):
        ret[i] = ret[i] ^ key[i%4]
    return ret

level_shift_right = {19:9, 17:11, 15:12, 12:16, 10:16, 9:18, 7:20, 5:22}
level_shift_left = {19:7, 17:5, 15:4, 12:0, 10:0, 9:0, 7:0, 5:0}

def xy_2_idx(x, y, level):
    shift = level_shift_right[level]
    x = x >> shift
    y = y >> shift
    c3 = (x & 0x7) | ((y & 0x7) << 3)
    c2 = ((x>>3) & 0x7) | (((y>>3) & 0x7) << 3)
    c1 = ((x>>6) & 0x7) | (((y>>6) & 0x7) << 3)
    shift_c0 = level_shift_left[level]
    x = x >> 9
    y = y >> 9
    c0 = x | ( y << shift_c0 )
    return c0, c1, c2, c3

def idx_2_xy(idx, level):
    c0, c1, c2, c3 = idx
    t = 2**level_shift_left[level] - 1
    n = c0 & t
    n = (c1 & 0x7) | (n << 3)
    n = (c2 & 0x7) | (n << 3)
    n = (c3 & 0x7) | (n << 3)
    n = n << level_shift_right[level]
    x = n
    c0 = c0 >> level_shift_left[level]
    c1 = c1 >> 3
    c2 = c2 >> 3
    c3 = c3 >> 3
    n = c0 & t
    n = (c1 & 0x7) | (n << 3)
    n = (c2 & 0x7) | (n << 3)
    n = (c3 & 0x7) | (n << 3)
    n = n << level_shift_right[level]
    y = n
    return x, y

powers = {(1<<i):1 for i in range(17)}

def get_layer_by_id(block, id):
    for layer in block.layer:
        if layer.layerId == id:
            return layer

trick = [0]
        

class VMap():
    def __init__(self, filename='/tmp/shanghai_289.dat'):
        self.dz = zlib.decompressobj()
        content = open(filename, 'rb').read()
        self.content = content
        c = content
        self.size_all = len(c)
        s = struct.unpack('i', c[0x20:0x24])[0]
        self.size_h0 = 0x20 + 4 + s
        h1 = c[self.size_h0:self.size_h0+0x100]
        h1 = decode_1(h1)
        p = self.size_h0+0x100
        self.level_count = struct.unpack('i', h1[0x50:0x54])[0]
        self.size_trees = struct.unpack('i', h1[0x9C:0x9C+4])[0]
        levels = []
        self.levels = levels
        self.size_h1 = 0x100
        self.size_h2 = 0x40 * self.level_count
        h2 = c[p:p+self.level_count*0x40]
        h2 = decode_1(h2)
        self.levels_info = {}
        for i in range(self.level_count):
            levels.append(struct.unpack('h', h1[0x58+8*i:0x58+2+8*i])[0])
            root = struct.unpack('i', h2[0x40*i+0x10:0x40*i+0x10+4])[0]
            self.levels_info[levels[i]] = {}
            self.levels_info[levels[i]]['root'] = root
            s00, s01, s10, s11, s20, s21, s30, s31 = struct.unpack('BBBBBBBB', h2[0x40*i:0x40*i+8])
            assert s00 == s01 and s10 == s11 and s20 == s21 and s30 == s31
            self.levels_info[levels[i]]['idx_block_size'] = s00*s01, s10*s11, s20*s21, s30*s31 # idx item size not byte size
        for i in range(self.level_count-1):
            self.levels_info[levels[i]]['end'] = self.levels_info[levels[i+1]]['root']
        self.levels_info[levels[-1]]['end'] = self.size_h1+self.size_h2+self.size_trees
        h = self.size_h0 + self.size_h1 + self.size_h2
        self.idx_part = h1+h2+decode_1(c[h:h+self.size_trees-4])+c[h+self.size_trees-4:h+self.size_trees]
        
    def get_block_pos(self, loc):
        header = self.content[loc+0x2f:loc+0x2f+0x10]
        key, compressed_len, origin_len = decode_block_header(header)
        block = self.content[loc+0x2f+0x10:loc+0x2f+0x10+compressed_len]
        block = decode_block(block, key)
        dz = zlib.decompressobj()
        block = dz.decompress(block)[:origin_len]
        return block            

    def get_block_xy(self, xy, level):
        c0, c1, c2, c3 = xy_2_idx(xy[0], xy[1], level)
        n0 = self.levels_info[level]['root']
        p = n0 + c0*4
        n1 = struct.unpack('i', self.idx_part[p:p+4])[0]
        p = n1 + c1*4
        n2 = struct.unpack('i', self.idx_part[p:p+4])[0]
        p = n2 + c2*4
        n2 = struct.unpack('i', self.idx_part[p:p+4])[0]
        p = n2 + c3*4
        n3 = struct.unpack('i', self.idx_part[p:p+4])[0]
        header = self.content[n3+0x2f:n3+0x2f+0x10]
#         return header
        key, compressed_len, origin_len = decode_block_header(header)
        print(key, compressed_len, origin_len)
        block = self.content[n3+0x2f+0x10:n3+0x2f+0x10+compressed_len]
#         return block
        block = decode_block(block, key)
        assert block[:2] == b'\x78\x9C'
#         assert False
        dz = zlib.decompressobj()
        block = dz.decompress(block)[:origin_len]
        return block
        
    def __repr__(self):
        atts = {}
        for i in dir(self):
#             print(i)
            if i[0] == '_' or i in ('contentaa', ) :
                continue
            att = self.__getattribute__(i)
            if type(att) == bytes:
                continue
            if inspect.ismethod(att):
                continue
            atts[i] = att
#         print(type(atts))
#         print(atts.keys())
#         return 'gogogo'
        return '%s ==> %s'%('hello',atts.__repr__())
    
    def walk_idx_tree_bak(self, level):
        s0 = self.levels_info[level]['root']
        e0 = self.levels_info[level]['end']
        p0 = s0
        size_0 = 0
        size_1 = 0
        size_2 = 0
        size_3 = 0
        while True:
            v = struct.unpack('i', self.idx_part[p0:p0+4])[0]
            if v > 0:
                size_0 = v - s0
                assert size_0 in powers
                print('size_0', size_0)
                break
            p0 += 4
        e0 = s0 + size_0
        v0 = []
        for i in range(size_0>>2):
            v = struct.unpack('i', self.idx_part[s0+i*4:s0+i*4+4])[0]
            if v > 0: v0.append(i)
        if size_1 == 0 and len(v0) > 1:
            ii_0, ii_1 = s0 + v0[0]*4, s0 + v0[1]*4
            size_1 = struct.unpack('i', self.idx_part[ii_1:ii_1+4])[0] - struct.unpack('i', self.idx_part[ii_0:ii_0+4])[0]
            assert size_1 in powers
            print('size_1', size_1)
        for i_1 in v0:
            s1 = struct.unpack('i', self.idx_part[s0+i_1*4:s0+i_1*4+4])[0]
            p = s1
            if size_1 == 0:
                while True:
                    v = struct.unpack('i', self.idx_part[p:p+4])[0]
                    if v > 0:
                        size_1 = v - s1
                        assert size_1 in powers
                        print('size_1', size_1)
                        break
                    p += 4
            e1 = s1 + size_1
            v1 = []
            for i in range(size_1>>2):
                v = struct.unpack('i', self.idx_part[s1+i*4:s1+i*4+4])[0]
                if v > 0: v1.append(i)
            if size_2 ==0 and len(v1) > 1:
                ii_0, ii_1 = s1 + v1[0]*4, s1 + v1[1]*4
                size_2 = struct.unpack('i', self.idx_part[ii_1:ii_1+4])[0] - struct.unpack('i', self.idx_part[ii_0:ii_0+4])[0]
                assert size_2 in powers
                print('size_2', size_2)
            for i_2 in v1:
                s2 = struct.unpack('i', self.idx_part[s1+i_2*4:s1+i_2*4+4])[0]
                p = s2
                if size_2 == 0:
                    while True:
                        v = struct.unpack('i', self.idx_part[p:p+4])[0]
                        if v > 0:
                            size_2 = v - s2
                            assert size_2 in powers
                            print('size_2', size_2)
                            break
                        p += 4
                e2 = s2 + size_2
                v2 = []
                for i in range(size_2>>2):
                    v = struct.unpack('i', self.idx_part[s2+i*4:s2+i*4+4])[0]
                    if v > 0: v2.append(i)
                if size_3 == 0 and len(v2) > 1:
                    ii_0, ii_1 = s2 + v2[0]*4, s2 + v2[1]*4
                    size_3 = struct.unpack('i', self.idx_part[ii_1:ii_1+4])[0] - struct.unpack('i', self.idx_part[ii_0:ii_0+4])[0]
                    assert size_3 in powers
                    print('size_3', size_3)
                for i_3 in v2:
                    s3 = struct.unpack('i', self.idx_part[s2+i_3*4:s2+i_3*4+4])[0]
                    p = s3
                    if size_3 == 0:
                        while True:
                            v = struct.unpack('i', self.idx_part[p:p+4])[0]
                            if v > 0:
                                if v > self.levels_info[level]['end']:
                                    size_3 = self.levels_info[level]['end'] - s3
                                else:
                                    size_3 = v - s3
                                assert size_3 in powers
                                print('size_3', size_3)
                                break
                            p += 4
                    e3 = s3 + size_3
                    for i in range(size_3>>2):
                        v = struct.unpack('i', self.idx_part[s3+i*4:s3+i*4+4])[0]
                        if v > 0:
#                             print('###########> ', hex(i_1), i_1, i_2, i_3, i)
                            pass
                            if i_2 == 0x2b and i_3 == 0x16 and i == 0x3a:
                                print('###########> ', hex(i_1), i_1, i_2, i_3, i, hex(v))
            
    def walk_idx_tree(self, level, f = None):
        s0 = self.levels_info[level]['root']
        size_0, size_1, size_2, size_3 = self.levels_info[level]['idx_block_size']
        n = 0
        for i_0 in range(size_0):
            c0, = struct.unpack('i', self.idx_part[s0+i_0*4:s0+i_0*4+4])
            if c0 > 0:
                for i_1 in range(size_1):
                    c1, = struct.unpack('i', self.idx_part[c0+i_1*4:c0+i_1*4+4])
                    if c1 > 0:
                        for i_2 in range(size_2):
                            c2, = struct.unpack('i', self.idx_part[c1+i_2*4:c1+i_2*4+4])
                            if c2 >0:
                                for i_3 in range(size_3):
                                    c3, = struct.unpack('i', self.idx_part[c2+i_3*4:c2+i_3*4+4]); trick[0] = c3
                                    if c3 > 0:
                                        if n >= 500000:
                                            return
                                        else:
                                            pass
                                        try:
                                            block = self.get_block_pos(c3)
                                            bl.Clear()
                                            bl.ParseFromString(block)
                                            if f:
                                                xy = idx_2_xy((i_0, i_1, i_2, i_3), 19)
                                                f(bl, xy)
                                            layer = get_layer_by_id(bl, 8)
                                            if layer:
#                                                 print('---------------------------', c3)
                                                n += 1
    #                                         print(get_layer_by_id(bl, 8))
#                                                 print(layer)
                                        except:
                                            traceback.print_exc()
                                            print('Oops', c3, i_0, i_1, i_2, i_3)
#                                         print('###########> ', hex(i_0), i_0, i_1, i_2, i_3)
                                        if i_1 == 0x2b and i_2 == 0x16 and i_3 == 0x3a:
                                            print('###########> ', hex(i_0), i_0, i_1, i_2, i_3)
            
import map_pb2
bl = map_pb2.Block()

def every_2_bits(raw):
    ret = []
    for n in raw:
        ret.append(n>>6)
        ret.append( (n>>4)&3 )
        ret.append( (n>>2)&3 )
        ret.append( n&3 )
    return ret

def decode_ints_with_bitmap(raw, bitmap):
    ret = []
    bitmap = every_2_bits(bitmap)
    n = 0
    for i in bitmap:
        item = 0
        for j in range(i+1):
            h = raw[n]
            item = item + (h<<(8*j))
            n += 1
        ret.append(item)
        if n == len(raw):
            break
    return ret

def lastbit_sign(a):
    if a&1:
        return (a>>1)*-1
    else:
        return a>>1
    
def pg(a, base_x=0, base_y=0, scale=1):
    x0, y0, a = a[0], a[1], a[2:]
    ret = [(x0,y0)]
    while a:
        dx, dy, a = a[0], a[1], a[2:]
        ret.append((ret[-1][0]+dx, ret[-1][1]+dy))
#     if ret[-1][0] - x0 < 0.1 and ret[-1][1] - y0 < 0.1:
#         ret[-1] = x0, y0
    ret = [(base_x+x/10, base_y+y/10) for (x, y) in ret]
    return ret

import json

out_f = None

def f1(block, base_xy):
    x, y = base_xy
#     x, y = 0, 0
    layer = get_layer_by_id(bl, 8)
    if layer:
        for body in layer.body:
            for b2 in body.b2:
                val = decode_ints_with_bitmap(b2.bo.data, b2.bo.mask)
                val = [lastbit_sign(i) for i in val]
                val = pg(val, x, y, 10)
                if out_f:
#                     out_f.write('%d %d %d'% (trick[0], base_xy[0], base_xy[1]))
                    out_f.write(json.dumps({'outline':val, 'height':b2.bo.height}))
                    out_f.write('\n')
                else:
                    print(json.dumps(val))
            

def f_poi(block, base_xy):
    x, y = base_xy
    layer = get_layer_by_id(bl, 3)
    if layer:
        for body in layer.body:
            for b2 in body.b2:
                if out_f:
                    out_f.write(json.dumps({'tag':body.tag, 'poi':b2.poi.poi, 'location':(x+b2.poi.x/100, y+b2.poi.y/100)}, ensure_ascii=False))
                    out_f.write('\n')

In [39]:
list(range(10))/100

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [80]:
m1 = VMap()
# m1

out_f = open('/tmp/289_poi.json', 'w')

%time m1.walk_idx_tree(19, f_poi)

out_f.close()
# m1 = VMap('/tmp/quanguogailue.dat')
# m1.walk_idx_tree(10)


###########>  0x6b3 1715 43 22 58
CPU times: user 53.2 s, sys: 286 ms, total: 53.4 s
Wall time: 53.4 s


In [389]:
def decode_b(body):
    a2 = body + 16
    v13 = a2 - 4;
    v15 = (a2 - 21) >> 2;
    v16 = v15 + 1;
    v17 = ((v15 - 3) >> 2) + 1;
    v18 = 4 * v17;
    v23 = 16 * v17 + 16;
    if v16 == v18:
        end = 16*v17
    else:
        end = 16*v17 + 4
        if v23 + 4 < v13:
            end += 4
            if v23 + 8 < v13:
                end += 4
    return end

def decode_b2(body):
    end = body >> 2 << 2
    if end == body:
        end -= 4
    print( ((body -1) >> 2 << 2) - end )
    return end

for i in range(16, 32):
    print(decode_b(i), i>>2 <<2, i)

12 16 16
16 16 17
16 16 18
16 16 19
16 20 20
20 20 21
20 20 22
20 20 23
20 24 24
24 24 25
24 24 26
24 24 27
24 28 28
28 28 29
28 28 30
28 28 31


In [347]:
idx_2_xy((1715, 36, 62, 18), 19)

(13526016, 3568640)

In [73]:
out_f = None
b = m1.get_block_xy((13531904, 3647232), 19)
bl.Clear()
bl.ParseFromString(b)
bl

b'\xd3\x10\xff\xf8' 21829 37670


layer {
  layerId: 7
  body {
    tag: 1604
    b2 {
      bg {
        data: "\001p\002\001_\026\027,;\002\005.W,\007,,\007$\305\276"
        mask: "\020\000\000\000\000\000"
      }
    }
  }
  body {
    tag: 1601
    b2 {
      bg {
        data: ":\002D\025\t9=\240\001C>\023\006"
        mask: "@\000\000\000"
      }
    }
    b2 {
      bg {
        data: "\001\020\002\001?\001NLD@5@1@/2"
        mask: "\021\000\000\000"
      }
    }
  }
  body {
    tag: 1549
    b2 {
      bg {
        data: "\340\262\003\r\005\225\237AA\0011d_\212\211\010\001\n\002\246\266\2450\001\r\014\r\002\013\002"
        mask: "\020\000\000\000\000\020\000\000"
      }
    }
    b2 {
      bg {
        data: "T\003b\001\275\257\240\265Z\0016.80\001F\001\001%&IR\021\0101>"
        mask: "P\000\000\000\000\000\000"
      }
    }
    b2 {
      bg {
        data: "\300\002*\003\241k\035\027/+y\207jg\016\nJej{nn+2\n\020\004\026\004\014* 2AHD\010\016\002\022\005\024\207\3129%\'<:&!8"
        mask: "P\000\000

In [74]:
out_f = None
b = m1.get_block_pos(444831 )
bl.Clear()
bl.ParseFromString(b)
bl
# f1(b,(100000,80000))

layer {
  layerId: 7
  body {
    tag: 2513
    b2 {
      bg {
        data: "\000\004\350\001\365\026\t\001\014\033\001\r\353=\001\035\244\001.\\\002#\0016"
        mask: "PD\000D\000"
      }
    }
    b2 {
      bg {
        data: "\000\004\350\001\365\026\t\001\014\033\001\r\353=\001\035\244\001.\\\002#\0016"
        mask: "PD\000D\000"
      }
    }
  }
  attr_3: 2000
  attr_4: 0
}
layer {
  layerId: 15
  body {
    tag: 61229
    b2 {
      bg {
        data: "\274\374\000 \003a\t\001\370\241\001\001\370\003\005\033\330\035\006\320\003)\035\000\004\r\004<\tE\001\020\335h\002\031`\t\t\334q\002Y\005\274\301\003\264\034U\004,\035\371D\001\001"
        mask: "d%\025UQTU "
      }
    }
    b2 {
      bg {
        data: "\000 \003x\\\341\247i0\234\002\005\tD\245\254/\001\300\t"
        mask: "\225U\020"
      }
    }
    b2 {
      bg {
        data: "\3509\002p\363\341\010\371\002\360?m\276\230\305\0025Y\002\001\001\361\001\315\001\205\001\231\002\035\001\271C\211\024\321\016u\004\2

In [46]:
if abdsfo:
    pass

NameError: name 'abdsfo' is not defined

In [271]:
n = 0
p = 512
while p < 287232:
    v = struct.unpack('i', m1.idx_part[p:p+4])[0]
    if v > 287232:
        n += 1
    p += 4
n

48418

In [301]:
# hexdump.hexdump(m1.idx_part)

In [133]:
hex(317444 - 0x4000 - 0x1000 - 0x400 - 0x4), hex(58100 - 0x10*5)

('0x48400', '0xe2a4')

In [15]:
hex(309504), hex(0x4d804 +0x100 + 0x100 ), 0x4d804

('0x4da04', '0x4da04', 317444)

In [69]:
xy_2_idx((13490348 + 13491991)//2, (3585779 + 3584323)//2, 19)
idx_2_xy((1715, 43, 29, 21), 19)
xy = (13490348 + 13491991)//2, (3585779 + 3584323)//2
hexdump.hexdump(m1.get_block_xy(xy, 19))
print('-------------------------')
hexdump.hexdump(decode_2(m1.get_block_xy(xy, 12)))
# hexdump.hexdump(guess_decode(m1.get_glock_xy(xy, 19)))

decode_block_header(m1.get_block_xy(xy, 12))

00000000: F8 9E FF F8 58 91 FF F8  01 05 00 00 50 9D FF F8  ....X.......P...
-------------------------
00000000: 00 00 00 00 A0 0F 00 00  2C D5 01 00 56 96 00 00  ........,...V...


(b'\xcd\xce\xfd\xf8', 38486, 120108)

In [80]:
m1.get_block_xy(xy, 19)

b'\xf8\x9e\xff\xf8X\x91\xff\xf8\x01\x05\x00\x00P\x9d\xff\xf8'

In [302]:
# hexdump.hexdump(m1.get_block_xy(xy, 19))
# m1.get_block_xy(xy, 19)

In [303]:
# hexdump.hexdump(m1.get_block_xy(xy, 19))

In [107]:
cc = m1.get_block_xy(xy, 12)
import zlib


dd = dz.decompress(cc)
# len(dd)

b'\xcd\xce\xfd\xf8' 38486 120108


error: Error -3 while decompressing data: incorrect data check

In [101]:
open('/tmp/z.bin', 'wb').write(cc)

38486

In [104]:
dz = zlib.decompressobj()
u = dz.decompress(cc)
len(u)

120118

In [122]:
# hexdump.hexdump(cc)